## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

dom = cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE)

In [ ]:
def full_canny(gnh, gnl, image):
  X, Y = image.shape
  lista = []

  for i in range(1, X - 1):
    for j in range(1, Y - 1):
      if(gnh[i, j] == 1):
        lista.append([i, j])

  visited = np.zeros((X, Y))
  edges = np.zeros((X, Y))

  while(lista):
    element = lista.pop()
    x = element[0]
    y = element[1]
    if(visited[x, y] == 0):
      visited[x, y] = 1
      edges[x, y] = 1
      for m in range(x - 1, x + 2):
        for n in range(y - 1, y + 2):
          if(gnl[m, n] == 1):
            lista.append([m, n])

  return edges

In [ ]:
def canny(image, thr, tl):
  X, Y = image.shape
  img_blur = cv2.GaussianBlur(image, (5, 5), 0)

  S1 = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
  S2 = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

  Gx = cv2.filter2D(img_blur, -1, S1)
  Gy = cv2.filter2D(img_blur, -1, S2)
  amplitude = np.sqrt(Gx**2 + Gy**2)

  alfa = np.arctan2(Gy, Gx)
  alfa_deg = np.degrees(alfa)
  A, B = alfa_deg.shape
  for i in range(A):
    for j in range(B):
      if alfa_deg[i][j] < 0:
        alfa_deg[i][j] + 180

  kier = np.zeros((X,Y))

  for i in range (X):
    for j in range (Y):
      if (0 <= alfa_deg[i,j] < 22.5) or (157.5 <= alfa_deg[i,j] <= 180):
        kier[i,j] = 1
      elif (22.5 <= alfa_deg[i,j] < 67.5):
        kier[i,j] = 2
      elif (67.5 <= alfa_deg[i,j] < 112.5):
        kier[i,j] = 3
      elif (112.5 <= alfa_deg[i,j] < 157.5):
        kier[i,j] = 4
    
  gn = nonmax(kier, amplitude)
  gnh = gn>=thr
  gnl = np.where(np.logical_and(thr > gn, gn >= tl),1,0)
  return gnh, gnl


def nonmax(kier,amplitude):
    X,Y = kier.shape
    gn = np.zeros((X,Y))
    
    for i in range (1, X-1):
      for j in range (1, Y-1):
        if(kier[i,j] == 1):
          if(amplitude[i,j-1] > amplitude[i,j] or amplitude[i,j+1] > amplitude[i,j]):
            gn[i,j] = 0
          else:
            gn[i,j] = amplitude[i,j]
        elif(kier[i,j] == 2):
          if(amplitude[i+1,j-1] > amplitude[i,j] or amplitude[i-1,j+1] > amplitude[i,j]):
            gn[i,j] = 0
          else:
            gn[i,j]=amplitude[i,j]
        elif(kier[i,j] == 3):
          if(amplitude[i+1,j] > amplitude[i,j] or amplitude[i-1,j] > amplitude[i,j]):
            gn[i,j]=0
          else:
            gn[i,j] = amplitude[i,j]
        elif(kier[i,j] == 4):
          if(amplitude[i-1,j-1] > amplitude[i,j] or amplitude[i+1,j+1] > amplitude[i,j]):
            gn[i,j] = 0
          else:
            gn[i,j] = amplitude[i,j]
    return gn

In [ ]:
gnh, gnl = canny(dom, tl = 5, thr = 10)
img_full_canny = full_canny(gnh = gnh, gnl = gnl, image = dom)
img_cv2_canny = cv2.Canny(dom, 5, 10, None, 3, 1)

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize = (23, 6))
ax1.set_title("Obraz oryginalny")
ax1.imshow(dom, 'gray')
ax1.axis('off')
ax2.set_title("Obraz GNH")
ax2.imshow(gnh, 'gray')
ax2.axis('off')
ax3.set_title("Obraz po pełnym Cannym")
ax3.imshow(img_full_canny, 'gray')
ax3.axis('off')
ax4.set_title("Obraz po Cannym wbudowanym")
ax4.imshow(img_cv2_canny, 'gray')
ax4.axis('off')

W obrazie funkcji wbudowanej można zauważyć białe plamy ponad domem oraz dokładniej widac dachówki. Na wszystkich wynikach (oprócz obrazu oryginalnego) słabo widoczne są okna z lewej strony domu.